In [1]:
import os, re, sys, glob

import numpy as np
import pandas as pd

from bionlp.util import fs, io, func


DATA_PATH = './evntpred/orig'
# DATA_PATH = './evntpred/pubmedcbow'
# DATA_PATH = './evntpred/cncptappend'
# DATA_PATH = './evntpred/cncptannot'
PARDIR_SEP = True

# Evaluation

In [2]:
FILE_PATTERN = '*/perf_clf_0.xlsx'
fpattern = os.path.abspath(os.path.join(DATA_PATH, FILE_PATTERN))
seppattern = os.path.join(os.path.basename(os.path.abspath(os.path.join(fpattern.replace('*', '(.+?)'), os.pardir))), os.path.basename(fpattern))

dfs, cols = [], []
for fpath in glob.glob(fpattern):
    dfs.append(pd.read_excel(fpath))
    try:
        pardir = os.path.basename(os.path.abspath(os.path.join(fpath, os.pardir)))
        col = os.path.join(pardir, os.path.basename(fpath)) if PARDIR_SEP else os.path.basename(fpath)
        refined_col = re.search(seppattern, col).group(1)
        cols.append(refined_col.strip('\\') if sys.platform.startswith('win32') else refined_col)
    except Exception as e:
        print e
        cols.append(col)
    
df = pd.concat(dfs, axis=1)
df.columns = cols
df = df.reindex(sorted(df.columns), axis=1)
df.to_excel('%s.xlsx' % DATA_PATH)
df

,0,1,2,3,4,5,6,7,8,9
Accuracy,0.935567,0.951031,0.804124,0.979381,0.938144,0.984536,0.984536,0.976804,0.976804,1.000000
Precision,0.685185,0.266667,0.260274,0.758621,0.526316,1.000000,0.714286,0.666667,0.375000,1.000000
Recall,0.948718,0.266667,0.275362,0.814815,0.666667,0.222222,0.555556,0.500000,1.000000,1.000000
F score,0.795699,0.266667,0.267606,0.785714,0.588235,0.363636,0.625000,0.571429,0.545455,1.000000
Train time,16.218808,15.982892,16.038120,16.329490,16.960581,16.399349,16.456199,16.164037,15.063874,13.928417
Test time,0.059468,0.059122,0.065620,0.064096,0.061737,0.061445,0.058527,0.060222,0.061024,0.060047


# Cross Validation

In [3]:
DATA_CV_PATH = DATA_PATH + '-cv'
FILE_PATTERN = '*/*/perf_clf_0.xlsx'
KFOLD = 5
fpattern = os.path.abspath(os.path.join(DATA_CV_PATH, FILE_PATTERN))
seppattern = os.path.join(os.path.basename(os.path.abspath(os.path.join(fpattern.replace('*', '(.+?)'), os.pardir))), os.path.basename(fpattern))

dfs, cols = [], []
for fpath in glob.glob(fpattern):
    dfs.append(pd.read_excel(fpath))
    try:
        gpardir = os.path.basename(os.path.abspath(os.path.join(fpath, os.pardir, os.pardir)))
        col = os.path.join(gpardir, os.path.basename(fpath)) if PARDIR_SEP else os.path.basename(fpath)
        refined_col = re.search(seppattern, col).group(1)
        cols.append(refined_col.strip('\\') if sys.platform.startswith('win32') else refined_col)
    except Exception as e:
        print e
        cols.append(col)
lbnum = len(dfs) / KFOLD
kfold_offset = np.cumsum([0] + [KFOLD] * lbnum)
dfs = [pd.concat(dfs[kfold_offset[i]:kfold_offset[i+1]], axis=1).mean(axis=1) for i in range(lbnum)]
df = pd.concat(dfs, axis=1)
df.columns = [cols[kfold_offset[i]] for i in range(lbnum)]
df = df.reindex(sorted(df.columns), axis=1)
df.to_excel('%s.xlsx' % DATA_CV_PATH)
df

,0,1,2,3,4,5,6,7,8,9
Accuracy,0.917353,0.961500,0.846774,0.967942,0.975090,0.988119,0.984233,0.979224,0.976338,0.997571
Precision,0.689167,0.566667,0.590064,0.567428,0.739341,0.395000,0.328139,0.325397,0.719091,0.963636
Recall,0.829860,0.311111,0.636267,0.654473,0.736364,0.300000,0.488889,0.190000,0.626032,0.911111
F score,0.747469,0.386273,0.607555,0.596522,0.718269,0.277731,0.367843,0.236591,0.603370,0.930000
Train time,12.596543,15.476677,17.597890,17.327099,15.417773,15.672250,16.575073,17.385895,17.658419,17.345797
Test time,0.059941,0.059742,0.063851,0.063560,0.061921,0.061440,0.059693,0.062188,0.060873,0.061413
